In [4]:


import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark/spark-streaming-kafka-0-10_2.11:2.3.1 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8:2.3.1 pyspark-shell'
# '--packages org.apache.spark:spark-streaming-kafka_2.10:1.6.0 pyspark-shell'


from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

# spark = SparkSession\
#     .builder\
#     .appName("streaming")\
#     .getOrCreate()


sc = SparkContext(appName="streaming")
sc.setLogLevel("WARN")

# import pyspark
# pyspark.__version__

Exception: Java gateway process exited before sending its port number

In [ ]:
ssc = StreamingContext(sc, 10)


In [2]:
# import os
import json
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

In [3]:
kafkaStream = KafkaUtils.createStream(ssc, 'cdh57-01-node-01.moffatt.me:2181', 'spark-streaming', {'twitter':1})

NameError: name 'ssc' is not defined

In [11]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

NameError: name 'kafkaStream' is not defined

In [6]:
parsed.count().map(lambda x:'Tweets in this batch: %s' % x).pprint()

In [7]:
authors_dstream = parsed.map(lambda tweet: tweet['user']['screen_name'])

In [8]:
author_counts = authors_dstream.countByValue()
author_counts.pprint()

In [9]:
author_counts_sorted_dstream = author_counts.transform(\
  (lambda foo:foo\
   .sortBy(lambda x:( -x[1]))))
#   .sortBy(lambda x:(x[0].lower(), -x[1]))\
#  ))
author_counts_sorted_dstream.pprint()

In [10]:
top_five_authors = author_counts_sorted_dstream.transform\
  (lambda rdd:sc.parallelize(rdd.take(5)))
top_five_authors.pprint()

In [11]:
filtered_authors = author_counts.filter(lambda x:\
                                                x[1]>1 \
                                                or \
                                                x[0].lower().startswith('rm'))

filtered_authors.transform\
  (lambda rdd:rdd\
  .sortBy(lambda x:-x[1]))\
  .pprint()

parsed.\
    flatMap(lambda tweet:tweet['text'].split(" "))\
    .countByValue()\
    .transform\
      (lambda rdd:rdd.sortBy(lambda x:-x[1]))\
    .pprint()

In [13]:
ssc.start()
ssc.awaitTermination(timeout=10)

Py4JJavaError: An error occurred while calling o26.awaitTerminationOrTimeout.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/streaming/util.py", line 65, in call
    r = self.func(t, *rdds)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/streaming/dstream.py", line 631, in <lambda>
    self.func = lambda t, rdd: func(t, prev_func(t, rdd))
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/streaming/dstream.py", line 295, in <lambda>
    func = lambda t, rdd: oldfunc(rdd)
  File "<ipython-input-10-e9595b02588e>", line 2, in <lambda>
    (lambda rdd:sc.parallelize(rdd.take(5)))
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packages/pyspark/rdd.py", line 1375, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "/home/neal/miniconda3/envs/keras/lib/python3.7/site-packag

In [1]:
import kafka

# connect to kafka cluster
cluster= kafka.KafkaClient("ec2-52-38-117-4.us-west-2.compute.amazonaws.com:9092")
topic="cloud"
consumer_group = 'default_group'
consumer = kafka.SimpleConsumer(cluster, consumer_group, topic)

# consume all messages
for raw in consumer:
    msg = raw.message.value
    print(msg)

SyntaxError: invalid syntax (simple.py, line 54)

In [ ]:
# this code works on master spark node
import json

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils



sc = SparkContext(appName="streaming")
sc.setLogLevel("WARN")

ssc = StreamingContext(sc, 1)

kafkaStream = KafkaUtils.createDirectStream(
    ssc,
    ['cloud'],
    {'metadata.broker.list':'10.0.0.11:9092'}
)

#collected = kafkaStream.collect()
#kafkaStream.foreachRDD(handler)
def handler(message):
    records = message.collect()
    for r in records:
        #print(r[1])
        read = json.loads(r[1].decode('utf-8'))['text']
#         print(read)
    #print(records)
    return

kafkaStream.foreachRDD(handler)
ssc.start()
ssc.awaitTermination(timeout=180)

In [ ]:
# next step is to load pipeline -> 
# feed text -> 
# filter IDs by tags (if applicable) ->
# parallelize the code from redis ->
# cosine similarity
# return highest similarity
